In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../helper_functions/")

import recency_util_live

import sys
sys.path.append("/home/ray__/ssd/BERT/") # gpt model utils location
from gpt_feat_utils import GPT_Inference


[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
import pickle

## initialise artifacts

#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu") # gpt model location.
domain = "se"
artifacts_dir = "/home/ray__/ssd/minds/" + domain + "/recency/" # location which contains com_map, sent_dict, kp_entity_graph, gc, lc, entity and label_dict

# During call

In [4]:
def computing_groups(request, artifacts_dir):    
    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = recency_util_live.compute_groups_new_call(request, artifacts_dir, True)
    return group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc

In [5]:
def get_filtered_groups(group):
    filtered_group = {}
    count = 0
    for groupid, freq in sorted(group_ent_map_rank_lc.items(), key=lambda kv:kv[1], reverse=True):
        if count !=5:
            filtered_group[groupid] = group[groupid]
        else:
            pass

    for groupid, freq in sorted(group_ent_map_rank_gc.items(), key=lambda kv:kv[1], reverse=True):
        if count !=5:
            filtered_group[groupid] = group[groupid]
        else:
            pass
    return filtered_group
 

In [6]:
# for groupid, groupobj in filtered_group.items():
#     print ("\n\nGroupid: ", groupid, "\n\n")
#     seg_list = [seg["originalText"] for seg in groupobj]
#     print (*seg_list, sep="\n\n")

In [29]:
# lc = pickle.load(open("/home/ray__/ssd/minds/se/recency/lc.pkl","rb"))

# Post call - graph related updates

In [6]:
import sys
sys.path.append("/home/ray__/ssd/pos_tag/code/") # Custom pos tag location.
sys.path.append("/home/ray__/ssd/") # ner util code
import recency_util_post

#from bert_ner_utils_graph import BERT_NER
#from distilbert_pos_tagger import DistilBertPosTagger

#pos_tagger = DistilBertPosTagger("/home/ray__/ssd/pos_tag/model/Distilbert/","cpu")
#ner_model = BERT_NER('/home/ray__/ssd/bert-ner/',labels = ["O", "MISC", "MISC",  "PER", "PER", "ORG", "ORG", "LOC", "LOC"],device="cpu")

{"level": "info", "filename": "file_utils.py", "lineno": 38, "module": "file_utils", "ts": "2020-02-23T05:22:26.616760Z", "msg": "PyTorch version 1.1.0 available."}
{"level": "warning", "filename": "tokenization_openai.py", "lineno": 96, "module": "tokenization_openai", "ts": "2020-02-23T05:22:40.668507Z", "msg": "ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy."}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-23T05:22:40.853283Z", "msg": "Adding _start_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-23T05:22:40.854154Z", "msg": "Adding _delimiter_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-23T05:22:40.855058Z", "msg": "Adding _classify_ to the vocabulary"}


In [7]:
def update_mind_artifacts(filtered_group, artifacts_dir):
    master_paragraphs, master_ids = recency_util_post.get_grouped_segments(filtered_group)
    multi_label_dict = pickle.load(open(artifacts_dir + "label_dict.pkl","rb"))
    ent_feat_dict = pickle.load(open(artifacts_dir + "entity.pkl","rb"))

    ent_sent_dict, kp_sent_dict, label_dict, noun_list, entity_dict = recency_util_post.form_sentence_graph(master_paragraphs, master_ids, multi_label_dict)
    ent_single_label_dict, all_sent_dict = recency_util_post.form_single_label_dict(label_dict, ent_sent_dict, kp_sent_dict)
    kp_entity_graph = recency_util_post.get_base_graph(artifacts_dir)
    kp_entity_graph, node_list = recency_util_post.update_entity_nodes(kp_entity_graph, ent_sent_dict, multi_label_dict)
    kp_entity_graph, node_list = recency_util_post.update_kp_nodes(kp_entity_graph, ent_sent_dict, node_list, kp_sent_dict)
    kp_entity_graph = recency_util_post.update_edges(kp_entity_graph, node_list, all_sent_dict, artifacts_dir)
    kp_entity_graph = recency_util_post.update_kp_tokens(kp_entity_graph, node_list)
    ent_kp_graph = recency_util_post.store_final_graph(kp_entity_graph, artifacts_dir)
    entity_dict = recency_util_post.update_entity_feat_dict(ent_sent_dict, ent_feat_dict)
    return ent_sent_dict, entity_dict

In [50]:
# pickle.dump(kp_entity_graph, open("/home/ray__/ssd/minds/se/recency/kp_entity_graph.pkl","wb"))

In [9]:
import os
os.listdir("../recency_dataset/se/")

['sync_eng_2020_01_14.txt',
 'sync_eng_2020_01_08.txt',
 'sync_eng_2020_01_10.txt',
 'sync_eng_2020_01_20.txt',
 'sync_eng_2020_02_12.txt',
 'sync_eng_2020_02_10.txt',
 'sync_eng_2020_01_03.txt',
 'sync_eng_2020_01_16.txt']

# pipelined Testing

In [126]:
# test it on a meeting
entity_sent_dict = []
test_set = ['sync_eng_2020_02_12.txt']
#test_set = os.listdir("../recency_dataset/se/")
artifacts_dir = "/home/ray__/ssd/minds/" + domain + "/recency/"

for file in test_set:
    with open("../recency_dataset/se/"+file,'rb') as f:
        request = json.load(f)
        if isinstance(request, str):
            request = json.loads(request)

    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = computing_groups(request, artifacts_dir)
    filtered_group = get_filtered_groups(group)
    ent_sent_dict, entity_dict = update_mind_artifacts(filtered_group, artifacts_dir)
    entity_sent_dict.append(ent_sent_dict)
    #entity_sent_dict.append(update_mind_artifacts(filtered_group, artifacts_dir))
    com_map, gc, lc = recency_util_post.get_entity_artifacts(artifacts_dir)
    fv = recency_util_post.get_common_entities(com_map, entity_dict)
    fv_new = dict([('<ETHER>-'+ent, entity_dict['<ETHER>-'+ent]) for ent in ent_sent_dict.keys() if ent not in fv and '<ETHER>-'+ent in entity_dict.keys()])
    new_ent_placement = recency_util_post.get_most_similar_entities(fv_new, fv)
    agreed_communities = recency_util_post.get_agreable_communities(new_ent_placement, com_map)
    com_map, gc, lc = recency_util_post.update_communitiy_artifacts(agreed_communities, com_map, gc, lc)
    recency_util_post.dump_community_artifacts(com_map, gc, lc, entity_dict, artifacts_dir)

Using  01DAAQY88QZB19JQZ5PRJFR76Y  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 77, "ts": "2020-02-23T08:30:55.810378Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-02-23T08:30:55.815189Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-02-23T08:30:59.230043Z", "msg": "Request Sent"}
('But if I actively control is done and recent polls out and buy the book anything what I was doing there are one more drop down with the select domain is also done with a book meeting now the things which are is fortunate than I have to work on.', '2020-02-12T05:41:33Z', '65bb83952fb54409a4bb59bb707f1375', '088725f0-a5a4-468f-9dda-b8424c686e31') ('And I will be done with women in red team paperbased like we can go

{"level": "info", "filename": "grouper_segments.py", "lineno": 1041, "module": "grouper_segments", "edges before prunning": 198, "edges after prunning": 198, "modularity": 0.8082811072887829, "ts": "2020-02-23T08:31:00.157203Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
But if I actively control is done and recent polls out and buy the book anything what I was doing there are one more drop down with the select domain is also done with a book meeting now the things which are is fortunate than I have to work on.
And I will be done with women in red team paperbased like we can go till the dashboard screen once that is done and he said I can pick up from there reading the screen that we have currently and integrating the new apis and on.
I feel more so like we sure can then the dashboard screen and then I will take up the conference pre so like we can have one single flow.
Silent ignore thing is in staging right now will release it to production but like everythin

After Merging 16
{'14': [(33, 7), (21, 5), (14, 4)], '11': [(33, 6), (49, 5), (7, 2), (28, 2)], '7': [(28, 12), (21, 5), (50, 3), (11, 3)], '3': [(28, 19), (11, 6), (21, 5), (57, 3)], '2': [(28, 13), (33, 4), (1432, 2), (21, 2)], '10': [(21, 5), (11, 3)], '15': [], '12': [(28, 8), (30, 6), (49, 6), (33, 2), (11, 2)], '5': [(28, 4)], '6': [(49, 3), (602, 2)], '0': [], '13': [(28, 10), (33, 4), (30, 2), (49, 2)], '9': [(28, 7), (33, 6), (58, 5), (4, 3), (443, 2), (762, 2), (7, 2)], '8': [], '1': [(7, 4), (21, 3), (1201, 3), (186, 2), (11, 2), (357, 2), (30, 2), (49, 2), (28, 2)], '4': [(28, 7), (56, 3), (777, 2), (15, 2), (1724, 2)]}
writing the gc and lc update.
updating community artifacts
<ETHER>-World War Two
<ETHER>-Web Desktop
<ETHER>-Jitsi Lip Jitsi
<ETHER>-De Puerto
<ETHER>-Iphone 6 Plus
<ETHER>-Jitsi
<ETHER>-Iframe
<ETHER>-Iphone 6
<ETHER>-Ffvp
<ETHER>-Kate
<ETHER>-Janus
<ETHER>-Navigation
<ETHER>-Guam
<ETHER>-Retinol
<ETHER>-Emily


In [127]:
lc_copy = pickle.load(open("/home/ray__/ssd/minds/se/recency/lc.pkl","rb"))
gc_copy = pickle.load(open("/home/ray__/ssd/minds/se/recency/gc.pkl","rb"))
com_map_updated = pickle.load(open("/home/ray__/ssd/minds/se/recency/com_map.pkl","rb"))

In [128]:
clusters = {}
for ent, com in com_map_updated.items():
    if com in clusters.keys():
        clusters[com].append(ent)
    else:
        clusters[com] = [ent]

In [125]:
sorted(lc_copy.items(), key=lambda kv:sum(kv[1]), reverse=True)[:5]

[(28, [131, 91, 118, 80, 95]),
 (11, [22, 92, 18, 15, 24]),
 (49, [35, 56, 12, 13, 14]),
 (33, [44, 15, 14, 26, 24]),
 (7, [33, 12, 11, 39, 0])]

In [132]:
clusters[21]

['Google',
 'Twitter',
 'Linkedin',
 'Facebook',
 'Life',
 'Youtube',
 'Patreon',
 'Quora',
 'Hope',
 'Wiki',
 'Docs',
 'Noteworthy',
 'Instagram',
 'Techcrunch',
 'Social',
 'Noteworthy in Tech Newsletter',
 'Dropbox',
 'Webhooks',
 'Getambassadorio',
 'Email',
 'Medium',
 'Googles',
 'Slack',
 'Whatsapp',
 'Post',
 'Intranet',
 'Google Docs',
 'Blog',
 'Part Ii',
 'Guide',
 'Slackbot',
 'Emojis',
 'Reddit',
 'Webtask',
 'Sms',
 'Gmail',
 'Stackoverflow',
 'Linkedins',
 'Online',
 'Webhook',
 'Gsuite',
 'Faun',
 'Medium Andtwitter',
 'Onlineornot',
 'Ebi',
 'Emoji',
 'Tumblr',
 'Onedrive',
 'Slash',
 'Kudos',
 'Codebursts',
 'Thanks',
 'Follow',
 'Insight Fellows',
 'Slack Channel',
 'Wikis',
 'Hacker News',
 'Yld',
 'Edurekas Official Site',
 'Article',
 'Faq',
 'Inbox',
 'Tesla',
 'Discdiver',
 'Teslas',
 'Forrestbrazeal',
 'Webiny',
 'Hackathon',
 'Mailchimp',
 'Awesome Web Digest',
 'Sydney Software Crafters Meetup',
 'Koinex',
 'Slack Api',
 'Great Books',
 'Robinhood',
 'Webhook

In [129]:
sorted(lc_copy.items(), key=lambda kv: sum(kv[1]), reverse=True)[:5]

[(28, [91, 118, 80, 95, 84]),
 (11, [92, 18, 15, 24, 16]),
 (49, [56, 12, 13, 14, 18]),
 (33, [15, 14, 26, 24, 29]),
 (21, [21, 16, 3, 16, 25])]

In [122]:
'<ETHER>-Ether Engineering' in entity_dict.keys()

True

In [32]:
lc

{14: [17],
 7: [16],
 33: [34],
 28: [89],
 15: [4],
 1: [19],
 49: [19],
 21: [19],
 17: [3],
 57: [13],
 30: [2],
 186: [2],
 48: [11],
 11: [57],
 628: [2],
 689: [2],
 357: [4],
 468: [2],
 214: [2],
 1201: [3],
 510: [2],
 602: [2],
 3: [8],
 527: [2],
 768: [2]}

In [135]:
added_entities = [ list(sent_dict.keys()) for sent_dict in entity_sent_dict]
added_entities = [i for j in added_entities for i in j]

In [128]:
kp_entity_graph_cl = pickle.load(open("/home/ray__/ssd/minds/se/recency/kp_entity_graph.pkl","rb"))
kp_entity_graph_df = pickle.load(open("/home/ray__/ssd/minds/se/recency_df_unpoluted/kp_entity_graph.pkl","rb"))
com_map_updated = pickle.load(open("/home/ray__/ssd/minds/se/recency/com_map.pkl","rb"))

In [140]:
len(new_entity_list)

105

In [139]:
nodes_list = list(kp_entity_graph_df.nodes())
new_entity_list = []
print (len(added_entities))
for ent in added_entities:
    if ent not in nodes_list:
        if '<ETHER>-'+ent in com_map_updated.keys():
            print (ent, com_map_updated['<ETHER>-'+ent])
            new_entity_list.append(ent)
            pass
        else:
            print (ent)
            pass
        
        #print (ent, end=", ")
        if ent not in kp_entity_graph_cl.nodes():
            print ("<False>")

267
Iframe
AI 7
Aztecs 7
Nazi 7
Franklin 7
Coptic 7
Dekada 7
Karthik 7
Backfilling Tusk 7
Caldwells 7
Filtration 7
Arjun 7
Trisha 7
Semi 7
Ether Engineering 7
Bill Franklin 7
New Link 7
Beyonce
Slack Graph Slack 7
Entity Graph 7
Janus 7
Sufi 7
Sookie 7
Channel China 7
Dario 7
Karthik 7
Janus Video Bridge 7
Janice 7
Correct 7
Rod 7
Graph Esther 7
Sages 7
Broncos 7
Franklin 7
Society
Z1 7
Leo 7
Gothic
General Eric 7
Cullen 7
Yen
Ether Mines 7
Costas 7
Dialogue Blog 7
Going Away 7
Us Michelle 7
Never Went Away 7
Cho 7
Trenton 7
Calendar Api 7
Vegan 7
Darren 7
Auerbach Bombards 7
Google Aps 7
Kings 7
Chao Garden De Labarre the Della Della Person 7
Latest
C Corp 7
Fifa 7
Suzhou 7
World War Two 7
Web Desktop 7
Jitsi Lip Jitsi 7
De Puerto 7
Iphone 6 Plus 7
Iframe
Ffvp 7
Janus 7
Navigation
Guam 7
Retinol 7
Emily
Olivia 7
Franklin 7
Squall 7
Decade 7
Institute 7
Ciso
May Day 7
Bike 82 7
Esper
Trenton 7
Llama
Camera Dot 7
Spo Rt 7
Multiple 7
Fft
Zeke 7
Paschal 7
Makonnen Karthik 7
Lifeline 7
Tro

In [132]:
clusters = {}
for ent, com in com_map_updated.items():
    if com in clusters.keys():
        clusters[com].append(ent)
    else:
        clusters[com] = [ent]

In [133]:
clusters[7]

['First',
 '3',
 'Machine',
 'Amazon Linux Ami',
 'Application',
 'Cpu',
 'Services',
 '1',
 'Version 1',
 '50',
 '000',
 '300',
 'Nvidia',
 'Gpus',
 '000+',
 'Sla',
 '20',
 'T2',
 'Micro Instance',
 'Disk',
 'Service',
 'Pms',
 'Gpu',
 'Gcps',
 'A4',
 'Iot',
 'Kpis',
 'Simple',
 'Us East',
 'Virginia',
 'Us-east-1',
 'Transcoding',
 '9',
 '500',
 'Benchmark',
 'P1',
 'P2',
 'P3',
 '100',
 'Xfs',
 'Sydney',
 'B',
 'Pvms',
 'Pvm',
 'N1-standard-1',
 'P90',
 'P99',
 'Spot',
 'Usd',
 'Wikipedia',
 'Mongoose',
 '2xlarge',
 '250',
 'Language',
 'Us-west-2',
 'Ebs',
 'Store',
 'Internet of Things',
 '5',
 'Hello World',
 '0',
 '4',
 'Target',
 'Pin',
 'Example',
 'Com',
 'One',
 '6',
 '7',
 '10',
 'Control',
 'Ghz',
 'Applications',
 'Slas',
 'Ab',
 'Ads',
 'Aperture',
 'Pinner',
 'Pinners',
 'Pinterest',
 '25',
 'Monad',
 'Code-based',
 'Language-specific',
 'System',
 '15',
 'Ssd',
 'C5d',
 '4xlarge',
 'Iops',
 'Hello',
 'Ram',
 'Stack',
 'Pinterests',
 'Lightstep',
 'Engine',
 '13',
 'Cud

In [80]:
e_fv = pickle.load(open("/home/ray__/ssd/minds/se/recency/entity.pkl", "rb"))

In [95]:
from scipy.spatial.distance import cosine
t_sim = []
t_sim = [(e, 1-cosine(e_fv[e], e_fv["<ETHER>-Ether Engineering"])) for e in e_fv.keys()]

In [96]:
list(sorted(t_sim, key=lambda kv:kv[1], reverse=True))[:10]

[('N', 0.7033717632293701),
 ('3', 0.6879128813743591),
 ('1', 0.6854665875434875),
 ('Usd', 0.6810983419418335),
 ('E', 0.670531153678894),
 ('Self', 0.6653602719306946),
 ('Naples', 0.6644787788391113),
 ('P', 0.6593735814094543),
 ('Control + C', 0.657631516456604),
 ('Ps', 0.6469427943229675)]

In [106]:
com_map_updated['Ps']

582

In [92]:
entity_sent_dict[0]['Ether Engineering']

{'0_7': ['because it is a semi but if we fix this ether engineering model, we should be able to revert it back if the things are in line with their domain models.']}

In [84]:
kp_entity_graph_cl.nodes['Ether Engineering']

{'node_type': 'entity',
 'is_ether_node': True,
 'node_label': Counter({'O': 1, 'MISC': 0, 'PER': 0, 'ORG': 0, 'LOC': 0}),
 'ether_grp_ctr': 1,
 'ether_meet_ctr': 1,
 'ether_meet_freq_list': [1],
 'ether_sent_ctr': 1,
 'art_ctr': 0,
 'para_ctr': 0,
 'sent_ctr': 0}

In [83]:
e_fv['<ETHER>-Ether Engineering']

array([-5.66842020e-01, -8.86713147e-01,  2.84424722e-01, -1.68190882e-01,
       -8.32775176e-01, -5.42130582e-02, -3.82031471e-01,  2.51720458e-01,
       -1.49997309e-01,  2.09081471e-01,  3.08636874e-01,  6.06657937e-03,
       -4.24090177e-01,  1.00815976e+00,  4.18314151e-02,  5.16788721e-01,
        1.65952474e-01,  3.44545931e-01, -2.97805846e-01,  1.18614465e-01,
       -8.72313261e-01,  4.86476570e-01, -4.17368144e-01, -1.13776457e+00,
       -9.08986509e-01, -2.95945823e-01, -4.65909213e-01, -1.21076727e+00,
       -4.12040144e-01, -4.60889727e-01,  4.20498759e-01,  2.12508962e-01,
       -3.86505276e-01,  7.25064725e-02,  4.52879697e-01, -5.94648778e-01,
       -2.02315599e-01,  1.96110994e-01, -5.08287907e-01,  8.12984630e-02,
        4.94293183e-01,  9.89620447e-01,  7.43469372e-02, -8.45587626e-02,
       -8.56721222e-01,  5.03661811e-01,  7.95618057e-01, -4.85133588e-01,
        2.24094212e-01, -3.17098573e-02, -9.04043317e-01,  3.93434912e-01,
       -3.28572839e-02,  

In [69]:
[k for k in list(e_fv.keys()) if 'Ether Engineering' in k]

[]

In [ ]:
for index, (groupid, groupobj) in enumerate(list(filtered_group.items())[:5]):
    print ("\n\nGroupRank: ", index+1, " GroupId:  ", groupid ,"\n\n")
    seg_list = [seg["originalText"] for seg in groupobj]
    print (*seg_list, sep="\n\n")
    print ("\n")